In [1]:
VERSION

v"1.8.1"

In [2]:
ENV["COLUMNS"] = 1000
ENV["LINES"] = 20;

In [307]:
# using Pkg
# Pkg.add("BenchmarkTools")
# Pkg.update("BenchmarkTools")
# Pkg.build("BenchmarkTools")

In [310]:
using DataFrames
using GLMakie
using JLD2
using BenchmarkTools

using Revise
using Astroshaper
import SPICE

In [65]:
meta_kernel = "/Users/masanorikanamaru/Dropbox/spice/hera/kernels/mk/hera_study_PO_EMA_2024_shapeViewer.tm"

SPICE.furnsh(meta_kernel)

In [66]:
et_start = SPICE.utc2et("2027-02-09T22:00:00")
et_end   = SPICE.utc2et("2027-02-11T22:00:00")
step     = 3600/20

et_range = et_start:step:et_end

8.554824691849957e8:180.0:8.556552691849957e8

In [67]:
"""
Obtain a dataframe of ephemerides 

# Arguments
- `targ`   : Target body name
- `ets`    : AbstractVetor of observer epoch
- `ref`    : Reference frame of output position vector
- `abcorr` : Aberration correction flag
- `obs`    : Observing body name
"""
function spkpos_df(targ, ets::AbstractVector, ref, abcorr, obs)
    df = DataFrame(et=Float64[], x=Float64[], y=Float64[], z=Float64[], lt=Float64[])
    for et in ets
        pos, lt = SPICE.spkpos(targ, et, ref, abcorr, obs)  # pos [km], lt [s]
        pos = SPICE.convrt.(pos, "km", "m")
        push!(df, (et, pos..., lt))
    end
    df
end

function spkpos_vec(targ, ets::AbstractVector, ref, abcorr, obs)
    positions = Vector{Float64}[]
    for et in ets
        pos, lt = SPICE.spkpos(targ, et, ref, abcorr, obs)  # pos [km], lt [s]
        pos = SPICE.convrt.(pos, "km", "m")
        push!(positions, collect(pos))
    end
    positions
end


"""
# Arguments
- `from` : Name of the frame to transform from
- `to`   : Name of the frame to transform to
- `ets`  : Epoch of the rotation matrix

# Return
- `rotate` : A rotation matrix
"""
pxform(from, to, ets) = [SPICE.pxform(from, to, et) for et in ets];

In [68]:
sun_d1 = spkpos_vec("SUN",       et_range, "DIDYMOS_FIXED",   "None", "DIDYMOS")
sun_d2 = spkpos_vec("SUN",       et_range, "DIMORPHOS_FIXED", "None", "DIMORPHOS")
d1_d2  = spkpos_vec("DIDYMOS",   et_range, "DIMORPHOS_FIXED", "None", "DIMORPHOS")
d2_d1  = spkpos_vec("DIMORPHOS", et_range, "DIDYMOS_FIXED",   "None", "DIDYMOS");

In [69]:
D1_TO_D2 = pxform("DIDYMOS_FIXED",   "DIMORPHOS_FIXED", et_range)
D2_TO_D1 = pxform("DIMORPHOS_FIXED", "DIDYMOS_FIXED",   et_range)

D1_TO_J2000 = pxform("DIDYMOS_FIXED",   "J2000", et_range)
D2_TO_J2000 = pxform("DIMORPHOS_FIXED", "J2000", et_range);

In [70]:
SPICE.kclear()

In [71]:
fig = Figure()
ax = Axis3(fig[1, 1], aspect=:data)

# scatter!(0,0,0)
# scatter!(sun_d1.x, sun_d1.y, sun_d1.z, color=:orange)
# scatter!(d2_d1.x, d2_d1.y, d2_d1.z, color=:skyblue)
# scatter!(sun_d1.x, sun_d1.y, sun_d1.z, color=:orange)
# scatter!(d2_d1.x, d2_d1.y, d2_d1.z, color=:skyblue)

scatter!([r[1] for r in sun_d1], [r[2] for r in sun_d1], [r[3] for r in sun_d1], color=:orange, size=2)

display(fig)

GLMakie.Screen(...)

- まずはDidymosだけで熱計算してみる
- まずはDimorphosだけで熱計算してみる
- 二重小惑星として熱計算してみる

In [454]:
shapedir = "/Users/masanorikanamaru/Dropbox/spice/hera/kernels/dsk"

shapepath1 = joinpath(shapedir, "g_50677mm_rad_obj_dida_0000n00000_v001.obj")
shapepath2 = joinpath(shapedir, "g_06650mm_rad_obj_didb_0000n00000_v001.obj")

shape1 = ShapeModel(shapepath1; scale=1000, find_visible_facets=true, save_shape=false)
shape2 = ShapeModel(shapepath2; scale=1000, find_visible_facets=true, save_shape=false);

# Only Dimorphos

In [455]:
thermo_params = ThermoParams(  # [Michel+2016; Naidu+2020]
    A_B   = 0.059,  # Bolometric  Bond albedo
    A_TH  = 0.0,
    k     = 0.125,
    ρ     = 2170.,
    Cp    = 600.,
    ϵ     = 0.9,
    t_bgn = et_range[begin],
    t_end = et_range[end],
    Nt    = length(et_range),
    z_max = 0.6,
    Nz    = 41,
    P     = SPICE.convrt(DIDYMOS[:P], "hours", "seconds"),
)

Thermophysical parameters
-------------------------
A_B   : 0.059
A_TH  : 0.0
k     : 0.125
ρ     : 2170.0
Cp    : 600.0
ϵ     : 0.9
-------------------------
t_bgn : 8.554824691849957e8
t_bgn : 105180.37410616312 (Normalized by period P)
t_end : 8.556552691849957e8
t_end : 105201.61962468656 (Normalized by period P)
Nt    : 961
Δt    : 180.0000000000556
Δt    : 0.022130748461919817 (Normalized by period P)
-------------------------
z_max : 0.6
z_max : 6.057014348123621 (Normalized by skin depth l)
Nz    : 41
Δz    : 0.015
Δz    : 0.15142535870309054 (Normalized by skin depth l)
-------------------------
P     : 8133.4800000000005
l     : 0.09905870541414048
Γ     : 403.4228550788862
λ     : 0.07680491551461667
-------------------------


In [456]:
savepath = "tmp.jld2"
run_TPM!(shape1, et_range, sun_d1, thermo_params, savepath)

jldopen(savepath, "r+") do file
    file["D1_TO_J2000"] = D1_TO_J2000
end;

In [457]:
# draw(shape1; data=:temperature, colormap=:vik)

In [458]:
data = load("tmp.jld2");

In [462]:
# data["shape"]
# data["et_range"]
# data["sun"]
# data["thermo_params"]
# data["surf_temps"]
# data["forces"]
# data["torques"]

# data["D1_TO_J2000"]

In [265]:
# shapeそのもの
# shape.force
# shape.torque
# surface_temperature(shape)
# 座標変換の行列

In [286]:
sun_d1

961-element Vector{Vector{Float64}}:
 [1.4721005358642877e11, -6.367281720735319e10, 9.959846949004263e9]
 [1.3696529692242667e11, -8.346162361196844e10, 9.961357158265778e9]
 [1.2407667415962964e11, -1.016396663197668e11, 9.962867357206055e9]
 [1.0879293360489598e11, -1.178560482751008e11, 9.964377545823746e9]
 [9.140905778719571e10, -1.3179773276984914e11, 9.965887724117495e9]
 [7.22605701556347e10, -1.4319558622803497e11, 9.96739789208594e9]
 [5.17170592591351e10, -1.518295735874496e11, 9.968908049727722e9]
 [3.0175045153152058e10, -1.575330061227661e11, 9.97041819704146e9]
 ⋮
 [7.182776447304222e10, -1.4444965129092883e11, 1.1395700877761848e10]
 [5.111469005591973e10, -1.5301156499906912e11, 1.1397200632423267e10]
 [2.941473534859727e10, -1.5862012790627322e11, 1.1398700375498959e10]
 [7.146744553741784e9, -1.61167046130744e11, 1.140020010698761e10]
 [-1.5259468651053112e10, -1.606031208608672e11, 1.140169982688798e10]
 [-3.737141722594735e10, -1.5693919873557858e11, 1.14031995351